In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:26<00:00,  2.24it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.749760e+12               0.0      0.444746
std    9.192323e+12               0.0      0.384930
min    5.592422e+02               0.0      0.258227
25%    7.800283e+02               0.0      0.324432
50%    8.790679e+02               0.0      0.366624
75%    9.952545e+02               0.0      0.438231
max    4.999513e+13               0.0      7.889956
Number of infeasible solution: 0


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 1000 # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 14252.06
Epoch 1, Validation Loss: 11353.40
Epoch 2, Validation Loss: 2458.08
Epoch 3, Validation Loss: 1542.39
Epoch 4, Validation Loss: 1084.99
Epoch 5, Validation Loss: 971.39
Epoch 6, Validation Loss: 1021.46
Epoch 7, Validation Loss: 926.33
Epoch 8, Validation Loss: 964.00
Epoch 9, Validation Loss: 826.47
Epoch 10, Validation Loss: 759.06
Epoch 11, Validation Loss: 764.51
Epoch 12, Validation Loss: 672.06
Epoch 13, Validation Loss: 747.75
Epoch 14, Validation Loss: 818.27
Epoch 15, Validation Loss: 803.62
Epoch 16, Validation Loss: 726.48
Epoch 17, Validation Loss: 669.91
Epoch 18, Validation Loss: 694.39
Epoch 19, Validation Loss: 753.30
Epoch 20, Validation Loss: 683.41
Epoch 21, Validation Loss: 813.46
Epoch 22, Validation Loss: 682.54
Epoch 23, Validation Loss: 855.66
Epoch 24, Validation Loss: 878.73
Epoch 25, Validation Loss: 669.50
Epoch 26, Validation Loss: 626.54
Epoch 27, Validation Loss: 661.81
Epoch 28, Validation Loss: 715.42
Epoch 29, Valida

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 61.33it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    565.135432          0.091505      0.002898
std     273.609467          1.290856      0.000726
min     210.354417          0.000000      0.001328
25%     336.175255          0.000000      0.002352
50%     511.836116          0.000000      0.003000
75%     747.684359          0.000000      0.003185
max    1617.493314         29.640963      0.008331
Number of infeasible solution: 15


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 1000 # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 11882.62
Epoch 1, Validation Loss: 11657.28
Epoch 2, Validation Loss: 1092.48
Epoch 3, Validation Loss: 1329.93
Epoch 4, Validation Loss: 1065.02
Epoch 5, Validation Loss: 872.44
Epoch 6, Validation Loss: 1442.29
Epoch 7, Validation Loss: 850.61
Epoch 8, Validation Loss: 824.83
Epoch 9, Validation Loss: 766.36
Epoch 10, Validation Loss: 892.28
Epoch 11, Validation Loss: 2147748.04
Epoch 12, Validation Loss: 665.02
Epoch 13, Validation Loss: 703.67
Epoch 14, Validation Loss: 1190.98
Epoch 15, Validation Loss: 1060.81
Epoch 16, Validation Loss: 791.68
Epoch 17, Validation Loss: 231511.21
Epoch 18, Validation Loss: 955.99
Epoch 19, Validation Loss: 716.73
Epoch 20, Validation Loss: 644.50
Epoch 21, Validation Loss: 771.72
Epoch 22, Validation Loss: 609.89
Epoch 23, Validation Loss: 1313338198528.00
Epoch 24, Validation Loss: 709.92
Epoch 25, Validation Loss: 742.58
Epoch 26, Validation Loss: 696.56
Epoch 27, Validation Loss: 992.11
Epoch 28, Validation Loss: 995.

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 61.16it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    610.819200          0.017875      0.002874
std     190.853377          0.342633      0.000721
min     319.190974          0.000000      0.000999
25%     478.882411          0.000000      0.002304
50%     583.233021          0.000000      0.002999
75%     688.061123          0.000000      0.003231
max    1831.013301          7.704545      0.007045
Number of infeasible solution: 4
